In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.cuda import FloatTensor, LongTensor

np.random.seed(42)

In [2]:
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

PAD_TOKEN = '[PAD]' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNKNOWN_TOKEN = '[UNK]' # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]' # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]' # This has a vocab id, which is used at the end of untruncated target sequences

VOCAB_SIZE = 50000

class Vocab(object):

  def __init__(self, vocab_file, max_size):
    self._word_to_id = {}
    self._id_to_word = {}
    self._count = 0 # keeps track of total number of words in the Vocab

    # [UNK], [PAD], [START] and [STOP] get the ids 0,1,2,3.
    for w in [UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
      self._word_to_id[w] = self._count
      self._id_to_word[self._count] = w
      self._count += 1

    # Read the vocab file and add words up to max_size
    with open(vocab_file, 'r') as vocab_f:
      for line in vocab_f:
        pieces = line.split()
        if len(pieces) != 2:
          print('Warning: incorrectly formatted line in vocabulary file: %s\n' % line)
          continue
        w = pieces[0]
        if w in [SENTENCE_START, SENTENCE_END, UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
          raise Exception('<s>, </s>, [UNK], [PAD], [START] and [STOP] shouldn\'t be in the vocab file, but %s is' % w)
        if w in self._word_to_id:
          raise Exception('Duplicated word in vocabulary file: %s' % w)
        self._word_to_id[w] = self._count
        self._id_to_word[self._count] = w
        self._count += 1
        if max_size != 0 and self._count >= max_size:
          print("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (max_size, self._count))
          break

    print("Finished constructing vocabulary of %i total words. Last word added: %s" % (self._count, self._id_to_word[self._count-1]))

  def word2id(self, word):
    if word not in self._word_to_id:
      return self._word_to_id[UNKNOWN_TOKEN]
    return self._word_to_id[word]

  def id2word(self, word_id):
    if word_id not in self._id_to_word:
      raise ValueError('Id not found in vocab: %d' % word_id)
    return self._id_to_word[word_id]

  def size(self):
    return self._count

  def write_metadata(self, fpath):
    print("Writing word embedding metadata file to %s..." % (fpath))
    with open(fpath, "w") as f:
      fieldnames = ['word']
      writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)
      for i in xrange(self.size()):
        writer.writerow({"word": self._id_to_word[i]})
        


def article2ids(article_words, vocab):
    ids = []
    oovs = []
    unk_id = vocab.word2id(UNKNOWN_TOKEN)
    for w in article_words.split():
        w = str(w)
        i = vocab.word2id(w)
        if i == unk_id: # If w is OOV
            if w not in oovs: # Add to list of OOVs
                oovs.append(w)
                oov_num = oovs.index(w) # This is 0 for the first article OOV, 1 for the second article OOV...
                #ids.append(vocab.size() + oov_num) # This is e.g. 50000 for the first article OOV, 50001 for the second...
        else:
            ids.append(i)
    return ids, oovs


def abstract2ids(abstract_words, vocab, article_oovs):
  ids = []
  unk_id = vocab.word2id(UNKNOWN_TOKEN)
  for w in abstract_words:
    i = vocab.word2id(w)
    if i == unk_id: # If w is an OOV word
      if w in article_oovs: # If w is an in-article OOV
        vocab_idx = vocab.size() + article_oovs.index(w) # Map to its temporary article OOV number
        #ids.append(vocab_idx)
      else: # If w is an out-of-article OOV
        ids.append(unk_id) # Map to the UNK token id
    else:
      ids.append(i)
  return ids

In [3]:
import struct
from tensorflow.core.example import example_pb2

def example_gen(filename):
    reader = open(filename, 'rb')
    examples = []
    while True:
        len_bytes = reader.read(8)
        if not len_bytes: break # finished reading this file
        str_len = struct.unpack('q', len_bytes)[0]
        example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
        e = example_pb2.Example.FromString(example_str)
        examples.append(e)
        
    for e in examples:  
        article_text = e.features.feature['article'].bytes_list.value[0]
        abstract_text = e.features.feature['abstract'].bytes_list.value[0]
        yield (article_text.decode('utf-8'), abstract_text.decode('utf-8'))
        

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
vocab = Vocab("finished_files/vocab", VOCAB_SIZE)









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: perisic


In [5]:
from decoder import *

%load_ext autoreload
%autoreload 2

In [6]:
from decoder import *

def build_decoder_model():
    master = DRAGNNDecoderMaster()
    embeddings_computer = EmbeddingComputer(VOCAB_SIZE, 1000)
    master.add_component_encoder(TBRU("embed", TaggerRecurrent("input", "embed", False), embeddings_computer, (1,), True).cuda())
    #master.add_component(TBRU("extractive", TaggerRecurrent("embed", "extractive"), TaggerComputer(1000, 1000), (1,), True).cuda())
    master.add_component_encoder(TBRU("rnn", RNNSolidRecurrent("embed", "rnn"), RNNSolidComputer(1000, 500), (1,), True).cuda())
    
    master.add_component_decoder(TBRU("decoder_embed", TaggerRecurrent(None, "decoder_embed", True),embeddings_computer, (1,), True).cuda())
    master.add_component_decoder(TBRU("decoder", LSTMEncoderRecurrent("rnn", "decoder_embed", "decoder", False), LSTMEncoderComputer(1000, 1000), (1,), True).cuda())
    master.add_component_decoder(TBRU("output", TaggerRecurrent("decoder", "output", False), TaggerComputer(1000, VOCAB_SIZE), (1,), True).cuda())
    return master

model = build_decoder_model()
#torch.cuda.memory_allocated()

In [7]:
def add_padding(articles):
    lens = [len(article) for article in articles]
    max_len = max(lens)
    
    for i in range(len(articles)):
        articles[i].extend([vocab.word2id(PAD_TOKEN)]*(max_len - len(articles[i])))
    return np.array(articles).T

def add_padding_for_tagging(articles, targets):
    lens = [len(article) for article in articles]
    max_len = max(lens)
    
    for i in range(len(articles)):
        targets[i].extend([0]*(max_len - len(articles[i])))
        articles[i].extend([vocab.word2id(PAD_TOKEN)]*(max_len - len(articles[i])))
    return np.array(articles).T, np.array(targets)

In [8]:
def calculate_mask(articles):
    mask = (articles == vocab.word2id(PAD_TOKEN))
    mask = np.logical_xor(mask, np.ones(articles.shape))
    return mask

def get_target(self, article, abstract):
    return [ int(i in abstract and i != vocab.word2id(UNKNOWN_TOKEN)) for i in article]

def push_abs_ptr(article, abstract, i, abs_ptr):
    while abs_ptr < len(abstract) and (not abstract[abs_ptr] in article[i+1:] 
                                               or abstract[abs_ptr] == vocab.word2id(SENTENCE_START)
                                               or abstract[abs_ptr] == vocab.word2id(SENTENCE_STOP)
                                               or abstract[abs_ptr] == vocab.word2id(UNKNOWN_TOKEN)):
        abs_ptr += 1
    return abs_ptr

class Batcher():
    
    def __init__(self, filename, batch_size):
        self.batch_size = batch_size
        generator = example_gen(filename)

        self.batches = []
        while True:
            articles = []
            targets = []
            for i in range(batch_size):
                try:
                    article_text, abstract_text = next(generator)
                    article_ids, _ = article2ids(article_text, vocab)
                    target, _ = article2ids(abstract_text, vocab)
                    articles.append(article_ids)
                    targets.append(target)
                except:
                    break
            if len(articles) == 0:
                break
            articles = add_padding(articles)
            targets = add_padding(targets)
            mask = calculate_mask(articles)
            self.batches.append( (articles, targets, mask) )
    
    def generator(self):
        for batch in self.batches:
            yield batch

In [ ]:
def calculate_bleu(result, target):
    
def calculate_bleu_ngramm(result, target, n):
    
def check_ngramm_in_string(ngramm, target):
    for i, token in enumerate(target):
        for token in 
        while target[i] == ngramm[k] or target[i] == vocab.word2id(UNKNOWN_TOKEN):
            
        if token == ngramm[0]:
            k = 1
            

In [9]:
def calc_f1(tp, fp, tn, fn):
    precision = tp/(fp + tp)
    recall = tp/(tp + fn)
    f1 = 2*precision*recall/(precision+recall)
    return f1

def precalc_f1(articles_tokens, articles, target):
    mask = calculate_mask(articles_tokens).T
    result = (articles > 0.5)
    #print(result[0])
    #print(articles.shape, target.shape, mask.shape)
    n_res = np.logical_not(result)
    n_tar = np.logical_not(target)
    tp = (result * target * mask).sum()
    fp = (n_res * target * mask).sum()
    tn = (n_res * n_tar * mask).sum()
    fn = (result * n_tar * mask).sum()
    return tp, fp, tn, fn

In [10]:
import math
import time

def do_epoch(model, criterion, data, batch_size, optimizer=None):  
    epoch_loss = 0.
    is_train = not optimizer is None
    model.train(is_train)

    with torch.autograd.set_grad_enabled(is_train):
        for i, (article_text, target, mask) in enumerate(data.generator()):
            X_batch, y_batch = LongTensor(article_text), LongTensor(target)
            inputs = InputLayerState("input", False, X_batch)
            targetLayer = InputLayerState("target", False, y_batch)
            logits = model.train_run(inputs, targetLayer)
            
            logits = logits.squeeze(-1)
            #print(logits.view(1, -1).shape)
            loss = criterion(logits.view(-1, logits.shape[-1]), y_batch.view(-1))
            epoch_loss += loss.item()
            
            if is_train:
                optimizer.zero_grad()
                loss.backward()
                #nn.utils.clip_grad_norm_(model.parameters(), 1.)
                optimizer.step()
            
            f1 = 0
            print('\r[{}]: Loss = {:.4f}, F1 = {:.4f}'.format(i, loss.item(), f1), end='')
                        
    return epoch_loss

def fit(model, criterion, optimizer, train_data, epochs_count=1, 
        batch_size=32, val_data=None, val_batch_size=None):
    if not val_data is None and val_batch_size is None:
        val_batch_size = batch_size
        
    for epoch in range(epochs_count):
        start_time = time.time()
        train_loss = do_epoch(model, criterion, train_data, batch_size, optimizer)
        f1 = 0
        output_info = '\rEpoch {} / {}, Epoch Time = {:.2f}s: Train Loss = {:.4f}: F1-Score = {:.4f}'
        if not val_data is None:
            val_loss = do_epoch(model, criterion, val_data, val_batch_size, None)
            f1 = 0
            epoch_time = time.time() - start_time
            output_info += ', Val Loss = {:.4f}'
            print(output_info.format(epoch+1, epochs_count, epoch_time, train_loss, f1, val_loss))
        else:
            epoch_time = time.time() - start_time
            print(output_info.format(epoch+1, epochs_count, epoch_time, train_loss, f1))

In [11]:
criterion = nn.CrossEntropyLoss(ignore_index = vocab.word2id(PAD_TOKEN)).cuda()
optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, epochs_count=50, batch_size=32, train_data=Batcher("finished_files/chunked/train_000.bin",20),
    val_data=None, val_batch_size=32)

Epoch 1 / 50, Epoch Time = 26.98s: Train Loss = 266.3462: F1-Score = 0.0000
Epoch 2 / 50, Epoch Time = 26.74s: Train Loss = 92.2464: F1-Score = 0.0000
Epoch 3 / 50, Epoch Time = 26.89s: Train Loss = 36.0664: F1-Score = 0.0000
Epoch 4 / 50, Epoch Time = 26.90s: Train Loss = 11.6985: F1-Score = 0.0000
Epoch 5 / 50, Epoch Time = 26.95s: Train Loss = 2.5301: F1-Score = 0.0000
Epoch 6 / 50, Epoch Time = 26.95s: Train Loss = 0.8077: F1-Score = 0.0000
Epoch 7 / 50, Epoch Time = 27.10s: Train Loss = 0.4264: F1-Score = 0.0000
Epoch 8 / 50, Epoch Time = 26.95s: Train Loss = 0.2831: F1-Score = 0.0000
Epoch 9 / 50, Epoch Time = 27.01s: Train Loss = 0.2286: F1-Score = 0.0000
Epoch 10 / 50, Epoch Time = 26.97s: Train Loss = 0.1916: F1-Score = 0.0000
Epoch 11 / 50, Epoch Time = 26.99s: Train Loss = 0.1642: F1-Score = 0.0000
Epoch 12 / 50, Epoch Time = 26.97s: Train Loss = 0.1428: F1-Score = 0.0000
Epoch 13 / 50, Epoch Time = 26.99s: Train Loss = 0.1257: F1-Score = 0.0000
Epoch 14 / 50, Epoch Time = 2

KeyboardInterrupt: 

In [ ]:
model.save_model("first_try_decoder")

In [ ]:
fit(model, criterion, optimizer, epochs_count=50, batch_size=32, train_data="finished_files/chunked/train_000.bin",
    val_data="finished_files/chunked/val_000.bin", val_batch_size=32)